In [ ]:
import json
import requests
import time

BASE_URL = "http://localhost:8080/search"
QUERIES_FILE = 'queries_train.json'

def evaluate_engine():
    with open(QUERIES_FILE, 'r') as f:
        queries_data = json.load(f)

    total_precision = 0
    latencies = []
    count = 0

    print(f"Starting Evaluation: Quality (>0.1) & Efficiency (<35s)")
    print("-" * 50)

    for query, true_ids in queries_data.items():
        try:
            start_time = time.time()
            response = requests.get(BASE_URL, params={'query': query, 'cache_bust': start_time}, timeout=40)
            duration = time.time() - start_time

            if response.status_code == 200:
                results = response.json()
                top_10_ids = [str(res[0]) for res in results[:10]]

                # Precision@10
                relevant_count = len(set(top_10_ids) & set(true_ids))
                p10 = relevant_count / 10.0

                total_precision += p10
                latencies.append(duration)
                count += 1

                print(f"Query: {query[:30]}... | P@10: {p10:.2f} | Time: {duration:.2f}s")
            else:
                print(f"Error in query '{query}': Status {response.status_code}")
        except Exception as e:
            print(f"Connection Error in query '{query}': {e}")

    avg_p10 = total_precision / count if count > 0 else 0
    max_latency = max(latencies) if latencies else 0
    avg_latency = sum(latencies) / len(latencies) if latencies else 0

    print("-" * 50)
    print(f"SUMMARY:")
    print(f"1. Quality (Avg P@10): {avg_p10:.4f} {'YES' if avg_p10 > 0.1 else 'NO'}")
    print(f"2. Efficiency (Max Time): {max_latency:.2f}s {'YES' if max_latency < 35 else 'NO'}")
    print(f"3. Avg Latency: {avg_latency:.2f}s")

if __name__ == "__main__":
    evaluate_engine()